### Heroes Of Pymoli Data Analysis
* Of the 1163 active players, the vast majority are male (84%). There also exists, a smaller, but notable proportion of female players (14%).

* Our peak age demographic falls between 20-24 (44.8%) with secondary groups falling between 15-19 (18.60%) and 25-29 (13.4%).  
-----

### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File sitting on root with code
file_to_load = "purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)
purchase_data.head()


,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


## Player Count

* Display the total number of players


In [3]:
#grouped by SN
all_users=pd.DataFrame(purchase_data["SN"])
#counted number of unique values
unique_users=all_users.nunique()
#placed in dataframe and printed
unique_users_df=pd.DataFrame({"Total Users": unique_users})
unique_users_df


,Total Users
SN,576


## Purchasing Analysis (Total)

* Run basic calculations to obtain number of unique items, average price, etc.


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame


In [4]:
#pulled just the item names, and counted for uniqueness
#do not understand how I could not get the correct value according to the example
items=purchase_data["Item Name"]
unique_items=items.nunique()

#Calculated the correct values
average=purchase_data["Price"].mean()
total=purchase_data["Price"].sum()
count=purchase_data["Purchase ID"].count()

#placed in dataframe
df_summary=pd.DataFrame([{"Number of Unique Items": unique_items, "Average Price": average, "Number of Purchases": count, "Total Revenue": total}])
#formatted the columns and placed in correct order before printings
df_summary["Average Price"]=df_summary["Average Price"].map("${:.2f}".format)
df_summary["Total Revenue"]=df_summary["Total Revenue"].map("${:.2f}".format)
df_summary=df_summary[["Number of Unique Items","Average Price", "Number of Purchases", "Total Revenue"]]
df_summary


,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,$3.05,780,$2379.77


## Gender Demographics

* Percentage and Count of Male Players


* Percentage and Count of Female Players


* Percentage and Count of Other / Non-Disclosed




In [5]:
#found this on stack and was quite useful
gender_df = purchase_data.groupby(['SN', 'Gender']).size().reset_index(name='Freq')
total=gender_df["SN"].count()
#placed counts in dataframe
gender_count_df=pd.DataFrame(gender_df["Gender"].value_counts())
#calculated formatted and altered column before print
gender_count_df["Percentage of Players"]=(gender_count_df["Gender"]/total) * 100
gender_count_df["Percentage of Players"]=gender_count_df["Percentage of Players"].map("{:.2f}%".format)
gender_count_renamed_df=gender_count_df.rename(columns={"Gender":"Total Count"})
gender_count_renamed_df



,Total Count,Percentage of Players
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%



## Purchasing Analysis (Gender)

* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender




* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [6]:
#pulled just male, female and nonmail/nonfemale dataframe
male_df=purchase_data[purchase_data.Gender=='Male']
female_df=purchase_data[purchase_data.Gender=='Female']
notmail_df=purchase_data[purchase_data.Gender!="Male"]
notfm_df=notmail_df[notmail_df.Gender!="Female"]

#placed all calculations and values into a datafram
gender_results_df=pd.DataFrame({"Gender": ["Male", "Female", "Other / Non-Disclosed"],
                              "Purchase Count": [male_df["Purchase ID"].count(), female_df["Purchase ID"].count(), notfm_df["Purchase ID"].count()],
                              "Average Purchase Price": [male_df["Price"].mean(), female_df["Price"].mean(), notfm_df["Price"].mean()],
                               "Total Purchase Value": [male_df["Price"].sum(), female_df["Price"].sum(), notfm_df["Price"].sum()],
                               "Avg Total Purchase per Person": [male_df["Price"].sum()/gender_count_renamed_df.loc["Male", "Total Count"], 
                                                                 female_df["Price"].sum()/gender_count_renamed_df.loc["Female", "Total Count"], 
                                                                 notfm_df["Price"].sum()/gender_count_renamed_df.loc["Other / Non-Disclosed", "Total Count"]]})
#formatted all the values before printing
gender_results_df["Average Purchase Price"]=gender_results_df["Average Purchase Price"].map("${:.2f}".format)
gender_results_df["Total Purchase Value"]=gender_results_df["Total Purchase Value"].map("${:.2f}".format)
gender_results_df["Avg Total Purchase per Person"]=gender_results_df["Avg Total Purchase per Person"].map("${:.2f}".format)
gender_results_df

,Gender,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
0,Male,652,$3.02,$1967.64,$4.07
1,Female,113,$3.20,$361.94,$4.47
2,Other / Non-Disclosed,15,$3.35,$50.19,$4.56


## Age Demographics

* Establish bins for ages


* Categorize the existing players using the age bins. Hint: use pd.cut()


* Calculate the numbers and percentages by age group


* Create a summary data frame to hold the results


* Optional: round the percentage column to two decimal points


* Display Age Demographics Table


In [7]:
#set bins and group labels
bins=[0,9,14,19,24,29,34,39,100]
group_age=["<10","10-14","15-19","20-24","25-29","30-34","35-39","40+"]
#once again from stack, very handy
age_df = purchase_data.groupby(['SN', 'Age']).size().reset_index(name='Freq')
#cut function
age_df["Age Group"] = pd.cut(age_df["Age"], bins, labels=group_age)
#created sub dataframe
ages_df=pd.DataFrame(age_df[["SN", "Age Group"]])
#rename columns
ages_df_renamed=ages_df.rename(columns={"SN": "Total Count"})
#created goupby and counted
ages_df_renamed=ages_df_renamed.groupby("Age Group")
ages_df2_renamed = pd.DataFrame(ages_df_renamed.count())
#calculated percentages, formatted and printed
ages_df2_renamed["Percentage of Players"]=(ages_df2_renamed["Total Count"] / int(unique_users)) * 100
ages_df2_renamed["Percentage of Players"]=ages_df2_renamed["Percentage of Players"].map("{:.2f}%".format)
ages_df2_renamed

,Total Count,Percentage of Players
Age Group,,
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


## Purchasing Analysis (Age)

* Bin the purchase_data data frame by age


* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [8]:
#created unitial dataframe
purch_df=purchase_data[['Age', 'Price']]
#cut via original bin information
purch_df["Age Group"]=pd.cut(purch_df["Age"], bins, labels=group_age)
#group by and inital calculations
purch_group=purch_df.groupby("Age Group")
purch_results_df=pd.DataFrame([purch_group["Age Group"].count(), 
                               purch_group["Price"].mean(), 
                               purch_group["Price"].sum()/ages_df2_renamed["Total Count"]])
#had to switch x,y axis labels
purch_results2_df=purch_results_df.transpose()
#renamed columns
purch_renamed_df=purch_results2_df.rename(columns={"Age Group": "Purchase Count", 
                                                "Price": "Average Purchase Price", 
                                                 "Unnamed 0":"Avg Total Purchase per Person"})
#added column
purch_renamed_df["Total Purchase Value"]=purch_group["Price"].sum()
#reorganized the columns
purch_reorg_df=purch_renamed_df[["Purchase Count", "Average Purchase Price", "Total Purchase Value", "Avg Total Purchase per Person"]]
#formatted the columns, and printed
purch_reorg_df["Purchase Count"]=purch_reorg_df["Purchase Count"].map("{:.0f}".format)
purch_reorg_df["Average Purchase Price"]=purch_reorg_df["Average Purchase Price"].map("${:.2f}".format)
purch_reorg_df["Total Purchase Value"]=purch_reorg_df["Total Purchase Value"].map("${:.2f}".format)
purch_reorg_df["Avg Total Purchase per Person"]=purch_reorg_df["Avg Total Purchase per Person"].map("${:.2f}".format)
purch_reorg_df



/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Age Group,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,$1114.06,$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


## Top Spenders

* Run basic calculations to obtain the results in the table below


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [9]:
#initial dataframe
spender=purchase_data.groupby("SN")
#inital calculations and had to transpose again
spender_df=pd.DataFrame([spender["SN"].count(), spender["Price"].sum()/spender["SN"].count(), spender["Price"].sum()])
spender2_df=spender_df.transpose()
#renamed columns, formatted, sorted, printed
spender_renamed_df=spender2_df.rename(columns={"SN":"Purchase Count", "Unnamed 0":"Average Purchase Price", "Price":"Total Purchase Value"})
spender_renamed_df["Purchase Count"]=spender_renamed_df["Purchase Count"].map("{:.0f}".format)
spender_renamed_df["Average Purchase Price"]=spender_renamed_df["Average Purchase Price"].map("${:.2f}".format)
spender_renamed_df["Total Purchase Value"]=spender_renamed_df["Total Purchase Value"].map("${:.2f}".format)
spender_sort_df=spender_renamed_df.sort_values("Purchase Count", ascending=False)
spender_sort_df.head()


,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Iral74,4,$3.40,$13.62
Idastidru52,4,$3.86,$15.45
Asur53,3,$2.48,$7.44
Inguron55,3,$3.70,$11.11


## Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns


* Group by Item ID and Item Name. Perform calculations to obtain purchase count, item price, and total purchase value


* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [10]:
#initial dataframe
item=purchase_data.groupby(["Item ID","Item Name"])
#initial calculations, and once again transposed
item_df=pd.DataFrame([item["Item Name"].count(), 
                      item["Price"].max(), 
                      item["Item Name"].count()*item["Price"].max()])
item_tran_df=item_df.transpose()
#columns renamed
item_renamed_df=item_tran_df.rename(columns={"Item Name":"Purchase Count",
                                            "Price": "Item Price", 
                                            "Unnamed 0":"Total Purchase Value"})
#formatted initally
item_renamed_df["Item Price"]=item_renamed_df["Item Price"].map("${:.2f}".format)
item_renamed_df["Total Purchase Value"]=item_renamed_df["Total Purchase Value"].map("${:.2f}".format)
#sorted prior to formatting column so it would sort based off int rather than string value
item_sort_df=item_renamed_df.sort_values("Purchase Count", ascending=False)
item_sort_df["Purchase Count"]=item_sort_df["Purchase Count"].map("{:.0f}".format)
item_sort_df.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
82,Nirvana,9,$4.90,$44.10
19,"Pursuit, Cudgel of Necromancy",8,$1.02,$8.16


## Most Profitable Items

* Sort the above table by total purchase value in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the data frame



In [11]:
#used a dataframe from previous example, renamed columns
item_renamed2_df=item_tran_df.rename(columns={"Item Name":"Purchase Count",
                                            "Price": "Item Price", 
                                            "Unnamed 0":"Total Purchase Value"})
#sorted initally so it would sort based off int value
item_sort2_df=item_renamed2_df.sort_values("Total Purchase Value", ascending=False)
#formatted and printed
item_sort2_df["Item Price"]=item_sort2_df["Item Price"].map("${:.2f}".format)
item_sort2_df["Total Purchase Value"]=item_sort2_df["Total Purchase Value"].map("${:.2f}".format)
item_sort2_df["Purchase Count"]=item_sort2_df["Purchase Count"].map("{:.0f}".format)
item_sort2_df.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
92,Final Critic,8,$4.88,$39.04
103,Singed Scalpel,8,$4.35,$34.80
